<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [94]:
#Load data
import pandas as pd
import numpy as np
data = pd.read_csv('TelcoCustomerChurn.csv')
data.shape

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-94-0fb19db988d5>", line 4, in <module>
    data = pd.read_csv('TelcoCustomerChurn.csv')
  File "/usr/local/lib/python3.6/dist-packages/pandas/io/parsers.py", line 685, in parser_f
    return _read(filepath_or_buffer, kwds)
  File "/usr/local/lib/python3.6/dist-packages/pandas/io/parsers.py", line 457, in _read
    parser = TextFileReader(fp_or_buf, **kwds)
  File "/usr/local/lib/python3.6/dist-packages/pandas/io/parsers.py", line 895, in __init__
    self._make_engine(self.engine)
  File "/usr/local/lib/python3.6/dist-packages/pandas/io/parsers.py", line 1135, in _make_engine
    self._engine = CParserWrapper(self.f, **self.options)
  File "/usr/local/lib/python3.6/dist-packages/pandas/io/parsers.py", line 1917, in __init__
    self._reader = parsers.TextReader(src, **kwds

FileNotFoundError: [Errno 2] File b'TelcoCustomerChurn.csv' does not exist: b'TelcoCustomerChurn.csv'

In [68]:
data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [69]:
#Looking for number of unique values in each of the columns
data.nunique()

customerID          7043
gender                 2
SeniorCitizen          2
Partner                2
Dependents             2
tenure                73
PhoneService           2
MultipleLines          3
InternetService        3
OnlineSecurity         3
OnlineBackup           3
DeviceProtection       3
TechSupport            3
StreamingTV            3
StreamingMovies        3
Contract               3
PaperlessBilling       2
PaymentMethod          4
MonthlyCharges      1585
TotalCharges        6530
Churn                  2
dtype: int64

In [70]:
binary_cols = ['Partner','Dependents','PhoneService','PaperlessBilling','Churn']
for col in binary_cols:
    data[col] = data[col].map({'Yes':1,'No':0})
data['gender'] = data['gender'].map({'Female':1,'Male':0})

In [71]:
data = data.fillna(0)

In [72]:
data.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [73]:
# Types of data in the dataframe
data.dtypes

customerID           object
gender                int64
SeniorCitizen         int64
Partner               int64
Dependents            int64
tenure                int64
PhoneService          int64
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling      int64
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                 int64
dtype: object

In [74]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
import category_encoders as ce

In [75]:
features = data.columns.drop(['customerID','Churn'])
X_train,X_test,y_train,y_test = train_test_split(data[features],data['Churn'],test_size=0.2,stratify=data['Churn'])
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((5634, 19), (5634,), (1409, 19), (1409,))

In [76]:
pipe = Pipeline([
        ('ce',ce.OneHotEncoder(use_cat_names=True)),
        ('rs',RobustScaler())
])

In [77]:
X_train = pipe.fit_transform(X_train)
X_test = pipe.transform(X_test)
y_train = y_train.values.reshape(len(y_train),1)
y_test = y_test.values.reshape(len(y_test),1)

In [78]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((5634, 40), (5634, 1), (1409, 40), (1409, 1))

In [82]:
from tensorflow.keras.optimizers import Adam
model = Sequential()
model.add(Dense(40,input_shape=(40,),activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(20, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='relu'))
model.add(Dropout(0.2))
# Output Layer
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
                    optimizer=Adam(lr=0.001), 
                    metrics=['accuracy'])
#model.summary()
history = model.fit(X_train, y_train, batch_size=40, epochs=500, validation_split=.1, verbose=1)

Train on 5070 samples, validate on 564 samples
Epoch 1/500
5070/5070 [==============================] - 2s 309us/sample - loss: 0.5281 - accuracy: 0.7258 - val_loss: 0.4792 - val_accuracy: 0.7163
Epoch 2/500
5070/5070 [==============================] - 1s 99us/sample - loss: 0.4840 - accuracy: 0.7499 - val_loss: 0.4690 - val_accuracy: 0.7535
Epoch 3/500
5070/5070 [==============================] - 0s 96us/sample - loss: 0.4689 - accuracy: 0.7746 - val_loss: 0.4609 - val_accuracy: 0.7819
Epoch 4/500
5070/5070 [==============================] - 0s 97us/sample - loss: 0.4584 - accuracy: 0.7763 - val_loss: 0.4561 - val_accuracy: 0.7748
Epoch 5/500
5070/5070 [==============================] - 0s 99us/sample - loss: 0.4509 - accuracy: 0.7868 - val_loss: 0.4574 - val_accuracy: 0.7819
Epoch 6/500
5070/5070 [==============================] - 1s 101us/sample - loss: 0.4402 - accuracy: 0.7945 - val_loss: 0.4524 - val_accuracy: 0.7819
Epoch 7/500
5070/5070 [==============================] - 1s 102

In [83]:

scores = model.evaluate(X_test, y_test)
print(f'{model.metrics_names[1]}: {scores[1]*100}')

1409/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?

In [93]:
grid_result.best_estimator_.predict(X_test)

1409/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

array([[0],
       [1],
       [0],
       ...,
       [0],
       [1],
       [0]])

In [84]:
#Now tring hyperparameter tuning

seed = 40
np.random.seed(seed)
# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(48, input_dim=40, activation='relu'))
    model.add(Dense(24,activation='relu'))
    model.add(Dense(24,activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=1)
param_grid = {'batch_size': [48,96],
              'epochs': [100,200]}
# Create Grid Search
grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_jobs=1,cv=5,verbose=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 4 candidates, totalling 20 fits
Train on 4507 samples
Epoch 1/100
4507/4507 [==============================] - 1s 167us/sample - loss: 0.5071 - accuracy: 0.7426
Epoch 2/100
4507/4507 [==============================] - 0s 60us/sample - loss: 0.4266 - accuracy: 0.7968
Epoch 3/100
4507/4507 [==============================] - 0s 60us/sample - loss: 0.4183 - accuracy: 0.7994
Epoch 4/100
4507/4507 [==============================] - 0s 60us/sample - loss: 0.4131 - accuracy: 0.8021
Epoch 5/100
4507/4507 [==============================] - 0s 61us/sample - loss: 0.4110 - accuracy: 0.8034
Epoch 6/100
4507/4507 [==============================] - 0s 61us/sample - loss: 0.4083 - accuracy: 0.8052
Epoch 7/100
4507/4507 [==============================] - 0s 63us/sample - loss: 0.4060 - accuracy: 0.8041
Epoch 8/100
4507/4507 [==============================] - 0s 63us/sample - loss: 0.4012 - accuracy: 0.8099
Epoch 9/100
4507/4507 [==============================] - 0s 63us/samp

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed: 11.0min finished


5634/5634 [==============================] - 1s 118us/sample - loss: 0.4955 - accuracy: 0.7494
Epoch 2/100
5634/5634 [==============================] - 0s 32us/sample - loss: 0.4242 - accuracy: 0.8037
Epoch 3/100
5634/5634 [==============================] - 0s 32us/sample - loss: 0.4161 - accuracy: 0.8049
Epoch 4/100
5634/5634 [==============================] - 0s 31us/sample - loss: 0.4130 - accuracy: 0.8053
Epoch 5/100
5634/5634 [==============================] - 0s 32us/sample - loss: 0.4107 - accuracy: 0.8108
Epoch 6/100
5634/5634 [==============================] - 0s 33us/sample - loss: 0.4076 - accuracy: 0.8104
Epoch 7/100
5634/5634 [==============================] - 0s 33us/sample - loss: 0.4079 - accuracy: 0.8108
Epoch 8/100
5634/5634 [==============================] - 0s 32us/sample - loss: 0.4032 - accuracy: 0.8131
Epoch 9/100
5634/5634 [==============================] - 0s 33us/sample - loss: 0.4018 - accuracy: 0.8147
Epoch 10/100
5634/5634 [==============================] -